### 抓取期交所PCR資料(Selenium)

In [3]:
import pandas as pd
import numpy as np
import requests
import os
import datetime as dt
from bs4 import BeautifulSoup

### 用requests的方式登入交易所網頁

In [7]:

startYear, endYear = 2025, 2025

df = date_dataframe(startYear=startYear, endYear=endYear)
lst = []
for k in range(df.shape[0]):
    url = 'https://www.taifex.com.tw/cht/3/pcRatio'
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',                 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',                     
    }

    payload = {
        'down_type':'',
        'queryStartDate':df.loc[k, 'startDate'],
        'queryEndDate': df.loc[k, 'endDate'],
    }


    res_j = requests.post(url, headers=headers, params=payload)
    res = BeautifulSoup(res_j.content, 'html.parser')
    tab = res.find_all('table', {'class':'table_f table-sticky table-fixed w-730'})

    n = len(tab[0].find_all('tr'))
    colName = [i.text for i in tab[0].find_all('tr')[0].find_all('th')]
    

    for i in range(1, n):
        row = [i.text.replace('\t', '').replace('\r', '').replace('\n', '') for i in tab[0].find_all('tr')  [i].find_all('td')]
        m = len(row)
        dd = {}
        for j in range(m):
            dd[colName[j]] = row[j]
        lst.append(dd)


    


df = pd.DataFrame(lst)

df['日期'] = pd.to_datetime(df['日期'])
df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace(',', '', regex=True)).astype(float)
df.sort_values(by='日期', inplace=True)

### 把登入的歷史時間用函數包裝

In [8]:
def date_dataframe(startYear, endYear):
    import calendar
    import datetime as dt
    import pandas as pd
    yrs = [i for i in range(startYear, endYear+1)]
    month = [i for i in range(1, 13)]

    lst_start = []
    lst_end = []

    for i in yrs:
        now = dt.datetime.now()
        yrs_now = now.year
        month_now = now.month
        if i != yrs_now:
            for j in month:
                start = dt.date(i, j, 1)
                end = dt.date(i, j, calendar.monthrange(i, j)[1])
                lst_start.append(start)
                lst_end.append(end)
        if i == yrs_now:
            for j in [k for k in range(1, month_now + 1)]:
                start = dt.date(i, j, 1)
                end = dt.date(i, j, calendar.monthrange(i, j)[1])
                lst_start.append(start)
                lst_end.append(end)

    df = pd.DataFrame()
    df['startDate'] = [i.strftime('%Y/%m/%d') for i in lst_start]
    df['endDate'] = [i.strftime('%Y/%m/%d') for i in lst_end]
    
    return df

    


### 資料呈現

In [9]:
print(df.to_string())

           日期     賣權成交量     買權成交量  買賣權成交量比率%    賣權未平倉量    買權未平倉量  買賣權未平倉量比率%
14 2025-01-02  393927.0  522422.0      75.40   85528.0  104662.0       81.72
13 2025-01-03  225515.0  238923.0      94.39  120744.0  139332.0       86.66
12 2025-01-06  400953.0  334280.0     119.95  173329.0  153175.0      113.16
11 2025-01-07  426990.0  437274.0      97.65  193571.0  177985.0      108.76
10 2025-01-08  537831.0  640380.0      83.99  101806.0  111995.0       90.90
9  2025-01-09  229746.0  296904.0      77.38  127763.0  160622.0       79.54
8  2025-01-10  182008.0  237397.0      76.67  141396.0  181146.0       78.06
7  2025-01-13  401252.0  574490.0      69.84  151933.0  221090.0       68.72
6  2025-01-14  353783.0  370662.0      95.45  170029.0  228683.0       74.35
5  2025-01-15  506261.0  569337.0      88.92   61242.0   71893.0       85.18
4  2025-01-16  228298.0  228283.0     100.01  108722.0  112016.0       97.06
3  2025-01-17  287036.0  366660.0      78.28  132315.0  137679.0       96.10